In [1]:
#Ruta a la carpeta que contiene los CSV's
RawData_rstate = '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM'

In [2]:
import os
import glob

# Obtener todos los archivos .csv en el directorio especificado
csv_files = glob.glob(os.path.join(RawData_rstate, '*.csv'))

# Imprimir la lista de archivos .csv
for file in csv_files:
    print(file)

/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_trovit_data_venta_Malaga_20240630.csv
/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_trovit_data_alquiler_Madrid_20240630.csv
/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_trovit_data_venta_Madrid_20240630.csv
/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_alquiler_madrid_20240630.csv
/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_venta_madrid_20240630.csv
/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TF

In [3]:
print(csv_files)

['/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_trovit_data_venta_Malaga_20240630.csv', '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_trovit_data_alquiler_Madrid_20240630.csv', '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_trovit_data_venta_Madrid_20240630.csv', '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_alquiler_madrid_20240630.csv', '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_venta_madrid_20240630.csv', '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master D

In [4]:
import pandas as pd

# Diccionario para almacenar los dataframes con sus nombres de archivo
dataframes = {}

# Ejemplo de lista de archivos CSV
csv_files = [
    '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_trovit_data_venta_Malaga_20240630.csv',
    '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_trovit_data_alquiler_Madrid_20240630.csv',
    '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_trovit_data_venta_Madrid_20240630.csv',
    '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_alquiler_madrid_20240630.csv',
    '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_venta_madrid_20240630.csv',
    '/Users/diegogloriasalamanca/Library/Mobile Documents/com~apple~CloudDocs/Master Data Science/99 TFM/TFM Inmobiliario/data/RawDataTFM/downloaded_data_venta_vizcaya_bizkaia_20240630.csv'
]

# Iterar sobre cada archivo CSV y crear un dataframe
for file in csv_files:
    # Extraer el nombre del archivo sin la ruta y la extensión
    file_name = file.split('/')[-1].replace('.csv', '')
    try:
        # Leer el archivo CSV en un dataframe de pandas
        df = pd.read_csv(file, delimiter=';', on_bad_lines='skip')
        
        # Determinar valor para columna 'fuente_datos'
        if 'trovit' in file_name:
            df['fuente_datos'] = 'trovit'
        else:
            df['fuente_datos'] = 'pisos.com'
        
        # Determinar valor para columna 'alquiler_venta'
        if 'alquiler' in file_name:
            df['alquiler_venta'] = 'alquiler'
        else:
            df['alquiler_venta'] = 'venta'
        
        # Determinar valor para columna 'fecha_extract'
        # Extraer la parte de la fecha del nombre del archivo
        fecha_extract_str = file_name.split('_')[-1]  # Obtener '20240630', '20240630', etc.
        # Convertir a formato de fecha
        fecha_extract = pd.to_datetime(fecha_extract_str, format='%Y%m%d')
        df['fecha_extract'] = fecha_extract
        
        # Almacenar el dataframe en el diccionario
        dataframes[file_name] = df
        
    except pd.errors.ParserError as e:
        print(f"Error parsing {file}: {e}")

# Ejemplo de acceso a uno de los dataframes y visualización
# Suponiendo que deseas visualizar el dataframe para 'downloaded_trovit_data_venta_Malaga_20240630'
print("Ejemplo de acceso y visualización:")
print(dataframes['downloaded_trovit_data_venta_Malaga_20240630'].head())

Ejemplo de acceso y visualización:
       precio                                          sub_descr  \
0  2.997.500€  Atico de lujo de 252 m2 en venta Torre del Rio...   
1  3.150.000€  Villa / Chalet de 513 m2 en venta en 29679 Ben...   
2    770.000€  Piso de lujo de 149 m2 en venta en Casco antig...   
3  4.500.000€  Piso de lujo de 137 m2 en venta en Puente Roma...   
4  6.995.000€  Piso de lujo de 137 m2 en venta en Puente Roma...   

                                                href  \
0  https://rd.clk.thribee.com/id.199191O1OC1h1k/o...   
1  https://rd.clk.thribee.com/id.1q1I1e1S51S91j/o...   
2  https://rd.clk.thribee.com/id.m_1yWq21o14/orig...   
3  https://rd.clk.thribee.com/id.U1M1Nt1JXu_/orig...   
4  https://rd.clk.thribee.com/id.Im1M1X1gc41k/ori...   

                       ubicacion      habitaciones    banios      mt2  \
0            Provincia de Málaga                 -  3  baños  252  m²   
1     29002, Provincia de Málaga  16  habitaciones  5  baños  513  m²   


In [5]:
print(dataframes)

{'downloaded_trovit_data_venta_Malaga_20240630':            precio                                          sub_descr  \
0      2.997.500€  Atico de lujo de 252 m2 en venta Torre del Rio...   
1      3.150.000€  Villa / Chalet de 513 m2 en venta en 29679 Ben...   
2        770.000€  Piso de lujo de 149 m2 en venta en Casco antig...   
3      4.500.000€  Piso de lujo de 137 m2 en venta en Puente Roma...   
4      6.995.000€  Piso de lujo de 137 m2 en venta en Puente Roma...   
...           ...                                                ...   
1420   2.150.000€  Vivienda de lujo de 160 m2 en venta Marbella, ...   
1421   3.495.000€  Villa / Chalet de lujo 746 m2 en venta, Marbel...   
1422   6.400.000€  Villa / Chalet de lujo de 641 m2 en venta en M...   
1423  12.250.000€  Villa / Chalet de 1229 m2 en venta en Marbella...   
1424   1.200.000€  Atico de alto standing de 233 m2 en venta Bena...   

                                                   href  \
0     https://rd.clk.thribe

In [6]:
import pandas as pd

# Suponiendo que `dataframes` es tu diccionario que contiene múltiples dataframes

# Convertir el diccionario de dataframes en una lista de dataframes
dfs = list(dataframes.values())

# Concatenar todos los dataframes en uno solo
consolidated_df = pd.concat(dfs, ignore_index=True)

# Mostrar el dataframe consolidado
print(consolidated_df.tail())

           precio                                sub_descr  \
4258  1.250.000 €                    Piso en Plaza Euskadi   
4259    135.000 €          Casa en Barrio de San Cristóbal   
4260    860.000 €             Piso en Avenida de Basagoiti   
4261    285.000 €                        Piso en Lasesarre   
4262    371.000 €  Piso en  Paraje Sur Isla de Zorrozaurre   

                                                   href  \
4258  /comprar/piso-abandoibarra_guggenheim48001-442...   
4259          /comprar/casa-ocharan-43349964457_999097/   
4260  /comprar/piso-getxo_algorta48991-42526550718_1...   
4261        /comprar/piso-lasesarre-44265195898_102200/   
4262  /comprar/piso-doneperiaga_deustuibarra_san_ped...   

                                              ubicacion habitaciones   banios  \
4258  Abandoibarra-Guggenheim (Distrito Abando. Bilbao)      3 habs.  2 baños   
4259                                    Ocharán (Zalla)      3 habs.   1 baño   
4260                         

In [7]:
consolidated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4263 entries, 0 to 4262
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   precio          4249 non-null   object        
 1   sub_descr       4263 non-null   object        
 2   href            4263 non-null   object        
 3   ubicacion       4263 non-null   object        
 4   habitaciones    4227 non-null   object        
 5   banios          4062 non-null   object        
 6   mt2             4254 non-null   object        
 7   otros           4262 non-null   object        
 8   publicado_hace  2850 non-null   object        
 9   plataforma      2850 non-null   object        
 10  CCAA            4263 non-null   object        
 11  fuente_datos    4263 non-null   object        
 12  alquiler_venta  4263 non-null   object        
 13  fecha_extract   4263 non-null   datetime64[ns]
 14  latitude        1413 non-null   object        
 15  long

In [8]:
import re
def limpiar_precio(precio):
    # Manejar valores NaN y convertir a cadena si es necesario
    if pd.isna(precio):
        return None
    # Convertir a cadena
    precio = str(precio)
    # Eliminar cualquier texto que no sea un número, punto, coma o espacio
    precio = re.sub(r'[^\d.,€\s]', '', precio)
    # Eliminar el símbolo de euro y espacios en blanco
    precio = re.sub(r'[€\s]', '', precio)
    # Reemplazar los puntos (separadores de miles) por nada
    precio = precio.replace('.', '')
    # Reemplazar las comas (separadores decimales) por nada
    precio = precio.replace(',', '')
    # Convertir a entero, manejar excepciones si la conversión falla
    try:
        return int(precio)
    except ValueError:
        return None

# Aplicar la misma transformación al dataframe consolidado
columns_to_clean_consolidated = ['precio', 'habitaciones', 'banios', 'mt2', 'publicado_hace' ] # Asegúrate de ajustar según tus columnas reales
for column in columns_to_clean_consolidated:
    consolidated_df[column] = consolidated_df[column].apply(limpiar_precio)

In [9]:
# Convertir las columnas 'latitude' y 'longitude' a tipo float
consolidated_df['latitude'] = pd.to_numeric(consolidated_df['latitude'], errors='coerce')
consolidated_df['longitude'] = pd.to_numeric(consolidated_df['longitude'], errors='coerce')

# Verificar el resultado
print(consolidated_df[['latitude', 'longitude']].dtypes)
print(consolidated_df[['latitude', 'longitude']].head())

latitude     float64
longitude    float64
dtype: object
   latitude  longitude
0       NaN        NaN
1       NaN        NaN
2       NaN        NaN
3       NaN        NaN
4       NaN        NaN


In [10]:
consolidated_df.describe()

,precio,habitaciones,banios,mt2,publicado_hace,fecha_extract,latitude,longitude
count,4.241000e+03,3650.000000,3782.000000,3788.000000,2850.000000,4263,1380.000000,1380.000000
mean,7.489121e+05,3.034521,2.356425,175.006864,305.033333,2024-06-30 00:00:00,40.869020,-3.601269
min,1.800000e+02,1.000000,1.000000,20.000000,1.000000,2024-06-30 00:00:00,40.025531,-4.345999
25%,1.300000e+04,2.000000,1.000000,87.000000,15.250000,2024-06-30 00:00:00,40.405420,-3.730088
50%,3.529000e+05,3.000000,2.000000,125.000000,30.000000,2024-06-30 00:00:00,40.436487,-3.688085
75%,8.950000e+05,4.000000,3.000000,201.000000,1119.000000,2024-06-30 00:00:00,40.543844,-3.586823
max,1.700000e+07,18.000000,14.000000,2982.000000,2040.000000,2024-06-30 00:00:00,43.420427,-2.484832
std,1.198029e+06,1.504870,1.310703,185.317958,490.129362,NaN,1.031325,0.300956


In [11]:
consolidated_df

,precio,sub_descr,href,ubicacion,habitaciones,banios,mt2,otros,publicado_hace,plataforma,CCAA,fuente_datos,alquiler_venta,fecha_extract,latitude,longitude,planta
0,2997500.0,Atico de lujo de 252 m2 en venta Torre del Rio...,https://rd.clk.thribee.com/id.199191O1OC1h1k/o...,Provincia de Málaga,NaN,3.0,252.0,...Residencial exclusivo situado en primera lí...,30.0,LuxuryEstate,Malaga,trovit,venta,2024-06-30,NaN,NaN,NaN
1,3150000.0,Villa / Chalet de 513 m2 en venta en 29679 Ben...,https://rd.clk.thribee.com/id.1q1I1e1S51S91j/o...,"29002, Provincia de Málaga",16.0,5.0,513.0,"...y zonas turísticas como San Pedro (8 km), P...",30.0,LuxuryEstate,Malaga,trovit,venta,2024-06-30,NaN,NaN,NaN
2,770000.0,Piso de lujo de 149 m2 en venta en Casco antig...,https://rd.clk.thribee.com/id.m_1yWq21o14/orig...,"Marbella, Provincia de Málaga",NaN,2.0,149.0,Último piso a la venta en un edificio en Casco...,30.0,LuxuryEstate,Malaga,trovit,venta,2024-06-30,NaN,NaN,NaN
3,4500000.0,Piso de lujo de 137 m2 en venta en Puente Roma...,https://rd.clk.thribee.com/id.U1M1Nt1JXu_/orig...,"Marbella, Provincia de Málaga",8.0,3.0,137.0,Moderno piso de 3 dormitorios en Puente Romano...,30.0,LuxuryEstate,Malaga,trovit,venta,2024-06-30,NaN,NaN,NaN
4,6995000.0,Piso de lujo de 137 m2 en venta en Puente Roma...,https://rd.clk.thribee.com/id.Im1M1X1gc41k/ori...,"Marbella, Provincia de Málaga",NaN,3.0,137.0,BlueLife Realty presenta a la venta este lujos...,30.0,LuxuryEstate,Malaga,trovit,venta,2024-06-30,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4258,1250000.0,Piso en Plaza Euskadi,/comprar/piso-abandoibarra_guggenheim48001-442...,Abandoibarra-Guggenheim (Distrito Abando. Bilbao),3.0,2.0,155.0,Este piso se encuentra en una zona privilegiad...,NaN,NaN,vizcaya_bizkaia,pisos.com,venta,2024-06-30,43.267567,-2.941757,8ª planta
4259,135000.0,Casa en Barrio de San Cristóbal,/comprar/casa-ocharan-43349964457_999097/,Ocharán (Zalla),3.0,1.0,298.0,"Se vende impresionante caserón en zalla, a tan...",NaN,NaN,vizcaya_bizkaia,pisos.com,venta,2024-06-30,43.216858,-3.122409,NaN
4260,860000.0,Piso en Avenida de Basagoiti,/comprar/piso-getxo_algorta48991-42526550718_1...,Algorta (Getxo),7.0,3.0,235.0,Inmobiliaria quorum getxo comercializa en excl...,NaN,NaN,vizcaya_bizkaia,pisos.com,venta,2024-06-30,43.350827,-3.011968,3ª planta
4261,285000.0,Piso en Lasesarre,/comprar/piso-lasesarre-44265195898_102200/,Lasesarre (Barakaldo),3.0,1.0,90.0,Espectacular vivienda de 90m² a 5 minutos de l...,NaN,NaN,vizcaya_bizkaia,pisos.com,venta,2024-06-30,43.298977,-2.985466,7ª planta


In [12]:
consolidated_df.to_csv('consolidated_data.csv', index=False)